In [47]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib auto

cv2.__version__

Using matplotlib backend: MacOSX


'3.1.0'

In [58]:
idxList = []
def onclick(event):
    print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          (event.button, event.x, event.y, event.xdata, event.ydata))
    global idxList
    idxList.append([event.xdata, event.ydata])

def getPoint(x,y,img):
    temp = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return temp[x,y]

def getLineSegment(x1,x2,y,img):
    temp = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return temp[x1:x2,y]

def colorFromIdx(idx, frame):
    x,y = idx 
    return frame[int(y), int(x), :]

def plotMask(idx, frame):
    idColor = colorFromIdx(idx, frame)
    offset = 20
    mask = np.zeros(frame.shape)
    for ii,color in enumerate(idColor):
        mask[:,:,ii] = (frame[:,:,ii] > color-offset) & (frame[:,:,ii]<color+offset)

    plt.figure()
    plt.imshow(np.sum(mask,axis=2))
    plt.title(str(idx))

def plotRefPoint(idx, ax):
    x,y = idx 
    ax.scatter(int(x), int(y))
    
def getMask(idColor, frame):
    offset = 20
    mask = np.zeros(frame.shape)
    for ii,color in enumerate(idColor):
        mask[:,:,ii] = (frame[:,:,ii] > color-offset) & (frame[:,:,ii]<color+offset) 
    mask = np.sum(mask, axis=2) ==3
    return mask

def getBlob(mask):
    blob = cv2.dilate(mask.astype(np.uint8), kernel)
    return blob

def getCentroid(blob):
    blob[blob==0] = 255
    blob[blob==1] = 0
    keypoints = detector.detect(blob)
    return keypoints

In [49]:
frameOffset = 300
bufLen = 5

# get these with mouse clicks later
x1 = 150
x2 = 470
y1 = 290
y2 = 450

# grab and display a frame during run
cap = cv2.VideoCapture('test.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES,frameOffset)
fs = cap.get(cv2.CAP_PROP_FPS)

ret,frame = cap.read()
frame = frame[x1:x2, y1:y2, :]
figRef = plt.figure()
axRef = figRef.add_subplot(111)
axRef.imshow(frame)

# mouse click on columns to histogram
cid = figRef.canvas.mpl_connect('button_press_event', onclick);

# Set up the SimpleBlobdetector with default parameters.
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 0;
params.maxThreshold = 256;

# Filter by Area.
params.filterByArea = True
params.minArea = 5

# Filter by Circularity
params.filterByCircularity = False
params.minCircularity = 0.1

# Filter by Convexity
params.filterByConvexity = False
params.minConvexity = 0.5

# Filter by Inertia
params.filterByInertia = False
params.minInertiaRatio = 0.5

detector = cv2.SimpleBlobDetector_create(params)


button=1, x=285, y=147, xdata=33.666667, ydata=237.000000
button=1, x=276, y=183, xdata=26.166667, ydata=207.000000
button=1, x=276, y=179, xdata=26.166667, ydata=210.333333
button=1, x=275, y=190, xdata=25.333333, ydata=201.166667


In [50]:
figRef.canvas.mpl_disconnect(cid);
    
for idx in idxList:    
    plotMask(idx,frame)
    plotRefPoint(idx,axRef)
    


In [52]:
DISP = False
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (30,30))
frameSkip = 0

# add user input to choose the best starting point
idx = idxList[1]
x,y = idx
idColor = colorFromIdx(idx, frame)
xBuff = [x]
yBuff = [y]

cap = cv2.VideoCapture('test.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES,frameSkip)
ret,frame = cap.read()
count = 0

while ret:
    count = count + 1
    frame =  frame[x1:x2, y1:y2, :]
    mask = getMask(idColor, frame)
    blob = getBlob(mask)
    
    keypoints = getCentroid(blob)
    
    # if we find any blobs
    if keypoints:
        
        # assume one and use that as the centroid
        x,y = keypoints[0].pt
        
        # if more than one find the one closest to the last one
        minVal = float('inf')
        for kp in keypoints:
            x,y = kp.pt
            dist = np.power((x-xBuff[-1]),2)+np.power((y-yBuff[-1]),2)
            if dist < minVal:
                minVal = dist
                xOut = x
                yout = y
                    
                
    else:
        
        # if no blob found then use the old value
        # mostly we get here because the blob touches the edge of the image
        if DISP:
            print('Bad')
        x = xBuff[-1]
        y = yBuff[-1]
        #input('')
        
    xBuff.append(x)
    yBuff.append(y)
    
    
    if DISP:
        new = cv2.circle(blob, (int(x),int(y)), 5, 255, -1)
        cv2.imshow('frame',new)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    ret,frame = cap.read()


cap.release()



True

In [53]:
cv2.destroyAllWindows()

# user input to figure out true offset
frameOffset = 500

# low pass filter to smooth
N=10
fc = fs/40.

import scipy.signal as sp

# use firwin with arbitrary Fs
h=sp.firwin( numtaps=N, cutoff=fc, nyq=fs/2)
y=sp.lfilter( h, 1.0, yBuff) # 'x' is the time-series data you are filtering


# count number of positive inflextions points (first derivative zero / second neg)
dy = np.diff(y)
dy[dy>0] = 1
dy[dy<0] = -1
ddy = np.diff(dy)
ddy[ddy>0] = 0
totalSteps = np.count_nonzero(ddy[frameOffset:25700])

print('Total Steps = %d'%totalSteps)

Total Steps = 765
